## Multi-Factor Portfolio Analysis Using CAPM, Fama-French, and Carhart Models

In [1]:

!pip install PyPortfolioOpt

#### Step 1: Install Necesary Libraries

In [2]:
import yfinance as yf
import pandas as pd
from pandas_datareader import data as web
from pypfopt import EfficientFrontier
from pypfopt import risk_models, expected_returns
import statsmodels.api as sm

#### Step 2: Download Stock Data

In [3]:
import yfinance as yf
import pandas as pd

tickers = ["JPM", "NVO", "AAPL", "MSFT", "XOM"]

prices = yf.download(tickers, start="2015-01-01", auto_adjust=True)["Close"]

# Convert to month-end prices
prices = prices.resample("M").last().dropna() #Monthly data is standard practice for factor modeling

prices.head()

[*********************100%***********************]  5 of 5 completed
/tmp/ipython-input-285000567.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  prices = prices.resample("M").last().dropna()


Ticker,AAPL,JPM,MSFT,NVO,XOM
Date,,,,,
2015-01-31,25.949120,40.474876,34.437153,17.747309,54.180443
2015-02-28,28.564180,45.610531,37.643963,19.017820,55.287643
2015-03-31,27.668077,45.089527,34.905437,21.597530,53.077129
2015-04-30,27.828167,47.397194,41.756042,22.762564,54.557056
2015-05-31,29.089870,49.285282,40.489403,22.847509,53.642075


#### Step 3: Compute Expected Returns and Covariance

In [4]:
# we tell PyPortfolioOpt that data is monthly (12 periods per year)
mu = expected_returns.mean_historical_return(prices, frequency=12) #this computes the returns internally
S = risk_models.sample_cov(prices, frequency=12)

# We are just computing this to get the weights so we give the prices dataframe so It can get the optimal weights

#### Step 4: Optimize Portfolio (Max Sharpe)

In [5]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

cleaned_weights

OrderedDict([('AAPL', 0.1514),
             ('JPM', 0.29276),
             ('MSFT', 0.53001),
             ('NVO', 0.0251),
             ('XOM', 0.00073)])

#### Step 5: Get Portfolio Returns (Monthly)

In [6]:
# Compute monthly returns for each stock
monthly_returns = prices.pct_change().dropna()

# Convert weights from dict to series
weights_series = pd.Series(cleaned_weights)

# Portfolio monthly return
portfolio_returns = monthly_returns @ weights_series # @ is for Matrix multiplication

portfolio_returns = portfolio_returns.to_frame("Portfolio_Return")

portfolio_returns.head()

# Now after getting the optimal weights for the porfolio, we now compute the monthly returns and multiply them by the weigths
# to get the portfolio return for each month
# example for february = 0.15(0.12)+0.29(0.08)+0.53(0.15)+0.02(0.10)+0.00073(0.05) = weight(return)

,Portfolio_Return
Date,
2015-02-28,0.103571
2015-03-31,-0.043275
2015-04-30,0.121254
2015-05-31,0.002531
2015-06-30,-0.028257


#### Step 6: Download Fama-French Monthly

In [7]:
from pandas_datareader import data as web

ff = web.DataReader("F-F_Research_Data_Factors", "famafrench", start="2015-01-01")[0]
ff = ff / 100
ff.index = ff.index.to_timestamp("M")

ff.head()

/tmp/ipython-input-1719030241.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = web.DataReader("F-F_Research_Data_Factors", "famafrench", start="2015-01-01")[0]
/tmp/ipython-input-1719030241.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = web.DataReader("F-F_Research_Data_Factors", "famafrench", start="2015-01-01")[0]


,Mkt-RF,SMB,HML,RF
Date,,,,
2015-01-31,-0.0310,-0.0059,-0.0345,0.0
2015-02-28,0.0613,0.0061,-0.0179,0.0
2015-03-31,-0.0111,0.0305,-0.0038,0.0
2015-04-30,0.0059,-0.0299,0.0180,0.0
2015-05-31,0.0137,0.0095,-0.0111,0.0


#### Step 7: Merge Portfolio + Fama French

In [8]:
df = portfolio_returns.merge(ff, left_index=True, right_index=True)

df["Excess_Return"] = df["Portfolio_Return"] - df["RF"] #new column (we dont use raw returns)

df.head()

,Portfolio_Return,Mkt-RF,SMB,HML,RF,Excess_Return
Date,,,,,,
2015-02-28,0.103571,0.0613,0.0061,-0.0179,0.0,0.103571
2015-03-31,-0.043275,-0.0111,0.0305,-0.0038,0.0,-0.043275
2015-04-30,0.121254,0.0059,-0.0299,0.0180,0.0,0.121254
2015-05-31,0.002531,0.0137,0.0095,-0.0111,0.0,0.002531
2015-06-30,-0.028257,-0.0152,0.0294,-0.0082,0.0,-0.028257


#### Step 8: CAPM on Optimized Portfolio

In [9]:
X_capm = sm.add_constant(df["Mkt-RF"])
y = df["Excess_Return"]

capm_model = sm.OLS(y, X_capm).fit()
print(capm_model.summary())

# Beta = 1: optimized portfolio moves almost exactly with the market. It’s basically market-level risk.
# Alpha ≈ 0.0087 per month (almost 10% per year), and is significant, it is strong. Means the
# portfolio is delivering returns above what CAPM predicts.
# R² = 75%: The market explains 75% of portfolio movement. Diversification made the portfolio more systematic.

                            OLS Regression Results                            
Dep. Variable:          Excess_Return   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.748
Method:                 Least Squares   F-statistic:                     386.5
Date:                Fri, 20 Feb 2026   Prob (F-statistic):           1.26e-40
Time:                        03:04:01   Log-Likelihood:                 291.93
No. Observations:                 131   AIC:                            -579.9
Df Residuals:                     129   BIC:                            -574.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0087      0.002      3.709      0.0

#### Step 9: Fama-French on Optimized Portfolio

In [10]:
X_ff = sm.add_constant(df[["Mkt-RF", "SMB", "HML"]])

ff_model = sm.OLS(y, X_ff).fit()
print(ff_model.summary())

# Beta = 1.07, Portfolio is still heavily market-driven.
# SMB (Small - Big) is negative and significant: portfolio loads toward large-cap stocks.
# HML (High - Low): Portfolio tilts slightly toward growth instead of value. Not very strong statistically.
# Alpha = 0.0074: Portfolio is delivering returns not fully explained by these 3 factors (abnormal).
# R² = 78.7% of the portfolio’s monthly returns are explained by the market, size (SMB), and value (HML) factors.

                            OLS Regression Results                            
Dep. Variable:          Excess_Return   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.782
Method:                 Least Squares   F-statistic:                     156.3
Date:                Fri, 20 Feb 2026   Prob (F-statistic):           1.90e-42
Time:                        03:04:01   Log-Likelihood:                 302.43
No. Observations:                 131   AIC:                            -596.9
Df Residuals:                     127   BIC:                            -585.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0074      0.002      3.329      0.0

- The optimized portfolio has a beta close to 1, meaning it carries market-level risk.
- Under CAPM, the market factor explains 75% of the portfolio’s returns, showing strong market dependence.
- The Fama-French model increases explanatory power to 78.7% (R² = 0.787), indicating that size and value factors improve the model slightly.
- The portfolio shows a negative and significant SMB loading, meaning it tilts toward large-cap stocks.
- The HML factor is weakly significant, suggesting a slight growth bias rather than strong value exposure.
- The portfolio alpha remains positive and statistically significant, indicating returns not fully explained by common risk factors.
- Diversification improved stability and factor clarity compared to analyzing a single stock.
- Overall, the optimized portfolio behaves like a systematic, large-cap, market-driven portfolio with moderate abnormal return potential.

#### Step 10: Carhart 4-Factor Model on Optimized Portfolio
- This is a 4 Factor model. we are doing it to compare to the CAPM and Fama French
- Momentum: The tendency of stocks that performed well in the recent past to continue performing well in the short term.

Download Momentum data

In [13]:
mom = web.DataReader("F-F_Momentum_Factor", "famafrench", start="2015-01-01")[0]
mom = mom / 100
mom.index = mom.index.to_timestamp("M")

mom.head()

/tmp/ipython-input-2355987427.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  mom = web.DataReader("F-F_Momentum_Factor", "famafrench", start="2015-01-01")[0]
/tmp/ipython-input-2355987427.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  mom = web.DataReader("F-F_Momentum_Factor", "famafrench", start="2015-01-01")[0]


,Mom
Date,
2015-01-31,0.0374
2015-02-28,-0.0310
2015-03-31,0.0270
2015-04-30,-0.0727
2015-05-31,0.0568


Merge df with Momentum
- the df we computed it above and it had (Portfolio_Return,	Mkt-RF, SMB, HML, RF, Excess_Return). now we add the momentum

In [17]:
df = df.merge(mom, left_index=True, right_index=True)
df.head()

,Portfolio_Return,Mkt-RF,SMB,HML,RF,Excess_Return,Mom
Date,,,,,,,
2015-02-28,0.103571,0.0613,0.0061,-0.0179,0.0,0.103571,-0.0310
2015-03-31,-0.043275,-0.0111,0.0305,-0.0038,0.0,-0.043275,0.0270
2015-04-30,0.121254,0.0059,-0.0299,0.0180,0.0,0.121254,-0.0727
2015-05-31,0.002531,0.0137,0.0095,-0.0111,0.0,0.002531,0.0568
2015-06-30,-0.028257,-0.0152,0.0294,-0.0082,0.0,-0.028257,0.0301


Carhart Regression

In [18]:
X = sm.add_constant(df[["Mkt-RF", "SMB", "HML", "Mom"]])

y = df["Excess_Return"]

carhart_model = sm.OLS(y, X).fit()
print(carhart_model.summary())

                            OLS Regression Results                            
Dep. Variable:          Excess_Return   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     116.5
Date:                Fri, 20 Feb 2026   Prob (F-statistic):           2.31e-41
Time:                        03:18:27   Log-Likelihood:                 302.55
No. Observations:                 131   AIC:                            -595.1
Df Residuals:                     126   BIC:                            -580.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0073      0.002      3.260      0.0

- The Momentum factor is not statistically significant, meaning the portfolio does not rely on recent winners or trend-following behavior.
- The portfolio still shows a positive and statistically significant alpha (~0.73% per month), meaning returns are not fully explained by market, size, value, or momentum factors.
- Adding momentum did not increase explanatory power, indicating the portfolio’s performance is not momentum-driven.